# MuMoT Demonstration Notebook: SEIR-Derived Epidemiological Model of COVID-19 and Social Distancing<a class="tocSkip">

## Multiscale Modelling Tool <a class="tocSkip">

*James A. R. Marshall, Department of Computer Science, University of Sheffield*

# Introduction
    


In [ ]:
import mumot

# Model Definitions

Our model is adapted from the standard SEIR model (Susceptiple, Exposed, Infectious, Recovered). The SEIR model is a model used to represent infectious diseases with a latent period following infection, before an individual becomes infectious, and has been used in modelling COVID-19 (Wu <i>et al.</i>, <a href='#references'>2020</a>). However, since COVID-19 also exhibits infection by asymptomatic individuals (Wu <i>et al.</i>, <a href='#references'>2020</a>), we also include an infection process from members of the $E$ class, which is not usual in SEIR models. Since asymptomatic individuals apparently can also recover directly without becoming symptomatic, we also adapt the SEIR model to allow direct recovery from the exposed class $E$. For simplicity, we assume the same infection rate $r$ regardless of whether a person is in $E$ or $I$.

For COVID-19 infection rates are assumed to be higher than recovery rates.

For health systems, the key population to track is the infectious population $I$, which in our model represents the symptomatic members of the population, since these are the ones that will require healthcare resources such as medicines, medical callouts, or hospitalisation. _Note that this model does not explicitly represent deaths from infection - all population members eventually recover, which is clearly not the case for COVID-19_

Thus our model is defined as follows:

In [ ]:
%%model
$
S + I -> I + E: r
S + E -> E + E: r
E -> I: a
E -> R: g
I -> R: g
$

In [ ]:
fullmodel = mumot.parseModel(In[-2])
fullmodel.show()

In [ ]:
finitemodel = fullmodel.substitute('S = N - I - E - R')

# Results

In the following, we analyse the models using both infinite-population ODEs, and finite population spatial stochastic simulations. _The latter allow us to vary *social isolation* and see the effects on the infection's progress_.

We parameterise the model to approximate the characteristics of COVID-19; the purpose of this demonstration is to show the effects of social distancing, so exact parameterisation is not required but could be found from the literature (<i>e.g.</i> Wu <i>et al.</i>, <a href='#references'>2020</a>). Here we choose $r=2$ and $a=g=1/2$. We then present a controller in which there is a single slider to change the degree of social isolation. This enables us to easily compare the effects of no social isolation (a 'well-mixed' population, left) with social isolation (a spatial, partially connected population, right). Open the 'Advanced options' tab and vary between interaction range between 0 (full social distancing) and 1 (no social distancing), then observe the effect on a sample infection trajectory in the right hand plot, compared to the left hand reference plot (no social distancing). <i>Note that social distancing not only reduces the peak number of cases needing medical attention, but also the total number of infections.</i>



In [ ]:
multicont1 = mumot.MuMoTmultiController(
    [finitemodel.integrate(showStateVars=['E', 'I', 'R'], ylab = 'cases', initialState = {'E': 0.05, 'I': 0.0, 'R': 0.0, 'S': 0.95}, maxTime = 10.0, plotProportions = True, silent = True, conserved = True, bookmark = False),
     finitemodel.multiagent(ylab = 'cases', initialState = {'E': 0.05, 'I': 0.0, 'R': 0.0, 'S': 0.95}, maxTime = 10.0, timestepSize = 0.25, randomSeed = 731529356, netType = 'dynamic', motionCorrelatedness = 0.5, particleSpeed = 0.01, showTrace = False, showInteractions = False, visualisationType = 'evo', plotProportions = True, realtimePlot = False, runs = 1, aggregateResults = True, silent = True, bookmark = False)], params = [('a', 0.5), ('g', 0.5), ('r', 2.0), ('plotLimits', 1), ('systemSize', 100.0)],
     choose_yrange = [0, 1])

# References 
<a id='references'></a>
<ul>
<li>Wu, J. T., Leung, K. and Leugn, G. M. (2020) Nowcasting and forecasting the potential domestic and international spread of the 2019-nCoV outbreak originating in Wuhan, China: a modelling study. <i>The Lancet</i> <b>395</b>, 689-697.
</ul>